In [17]:
import pyodbc

server = 'sqledge'  # This is the name of your Azure SQL Edge container
database = 'datasets'
username = 'sa'
password = 'bigStrfefongPwd4234#!#'

conn_str = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes;Encrypt=yes;'

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

query = """
SELECT 
    t.name AS TableName,
    s.name AS SchemaName
FROM 
    sys.tables t
INNER JOIN 
    sys.schemas s ON t.schema_id = s.schema_id
ORDER BY 
    s.name, t.name;
"""
cursor.execute(query)

# Fetch all results
tables = cursor.fetchall()

# Print the results
print("Tables in the database:")
for table in tables:
    print(f"Schema: {table.SchemaName}, Table: {table.TableName}")

# Close the cursor and connection
cursor.close()
conn.close()

Tables in the database:
Schema: dbo, Table: academic


In [5]:
import pyodbc


def query(query):
   

    server = 'sqledge'  # This is the name of your Azure SQL Edge container
    database = 'master'
    username = 'sa'
    password = 'bigStrfefongPwd4234#!#'

    conn_str = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes;Encrypt=yes;'

    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    
    cursor.execute(query)

    # Fetch all results
    tables = cursor.fetchall()

    # Close the cursor and connection
    cursor.close()
    conn.close()

    return tables


q = """
    SELECT 
        t.name AS TableName,
        s.name AS SchemaName
    FROM 
        sys.tables t
    INNER JOIN 
        sys.schemas s ON t.schema_id = s.schema_id
    ORDER BY 
        s.name, t.name;
    """

query(q)

[('MSreplication_options', 'dbo'),
 ('spt_fallback_db', 'dbo'),
 ('spt_fallback_dev', 'dbo'),
 ('spt_fallback_usg', 'dbo'),
 ('spt_monitor', 'dbo')]

In [7]:
import pandas as pd 
import polars as pl 

academic_df = pl.read_csv("backend/code_execution/data/academic.csv")
academic_df


year,students,us_students,undergraduate,graduate,non_degree,opt
str,i64,f64,f64,f64,f64,f64
"""1948/49""",25464,2.4034e6,null,null,null,null
"""1949/50""",26433,2.445e6,null,null,null,null
"""1950/51""",29813,2.281e6,null,null,null,null
"""1951/52""",30462,2.102e6,null,null,null,null
"""1952/53""",33675,2.134e6,null,null,null,null
…,…,…,…,…,…,…
"""2018/19""",1095299,1.9828e7,431930.0,377943.0,62341.0,223085.0
"""2019/20""",1075496,1.972e7,419321.0,374435.0,58201.0,223539.0
"""2020/21""",914095,1.9744e7,359787.0,329272.0,21151.0,203885.0


In [11]:
import pandas as pd 
import polars as pl 
import pyodbc

server = 'sqledge'  # This is the name of your Azure SQL Edge container
database = 'datasets'
username = 'sa'
password = 'bigStrfefongPwd4234#!#'

def get_sql_type(dtype):
    if dtype == pl.String:
        return 'NVARCHAR(MAX)'
    elif dtype == pl.Int64:
        return 'BIGINT'
    elif dtype == pl.Float64:
        return 'FLOAT'
    elif dtype == pl.Boolean:
        return 'BIT'
    elif dtype == pl.Datetime:
        return 'DATETIME2'
    else:
        return 'NVARCHAR(MAX)'  # Default to NVARCHAR(MAX) for unknown types


def add_table_from_csv(csv):
    df = pl.read_csv(f"backend/code_execution/data/{csv}")
    
    
    if len(df)> 1000:
        return

    print(len(df))
    
    columns = ["id INT IDENTITY(1,1) PRIMARY KEY"]
    for column, dtype in zip(df.columns, df.dtypes):
        sql_type = get_sql_type(dtype)
        columns.append(f"[{column}] {sql_type}")

    create_table_query = f"""
    CREATE TABLE {csv.replace('.csv','')} (
        {', '.join(columns)}
    )
    """
    print(create_table_query)

    conn_str = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes;Encrypt=yes;'
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    cursor.execute(create_table_query)
    conn.commit()
    # Prepare the batch insert query
    placeholders = ', '.join(['?' for _ in df.columns])
    insert_query = f"""
    INSERT INTO {csv.replace('.csv','')} ({', '.join([f'[{col}]' for col in df.columns])})
    VALUES ({placeholders})
    """

    # Convert the DataFrame to a list of tuples for batch insert
    data = [tuple(row) for row in df.iter_rows()]

    # Perform batch insert
    cursor.fast_executemany = True
    cursor.executemany(insert_query, data)
    conn.commit()
    cursor.close()
    conn.close()



In [12]:
import os 

for file in os.listdir("backend/code_execution/data/"):
    if '.csv' in file:
        add_table_from_csv(file)

75

    CREATE TABLE academic (
        id INT IDENTITY(1,1) PRIMARY KEY, [year] NVARCHAR(MAX), [students] BIGINT, [us_students] FLOAT, [undergraduate] FLOAT, [graduate] FLOAT, [non_degree] FLOAT, [opt] FLOAT
    )
    
1000

    CREATE TABLE student_performance (
        id INT IDENTITY(1,1) PRIMARY KEY, [gender] NVARCHAR(MAX), [race/ethnicity] NVARCHAR(MAX), [parental level of education] NVARCHAR(MAX), [lunch] NVARCHAR(MAX), [test preparation course] NVARCHAR(MAX), [math score] BIGINT, [reading score] BIGINT, [writing score] BIGINT
    )
    
29

    CREATE TABLE pytest_expected_output (
        id INT IDENTITY(1,1) PRIMARY KEY, [Name] NVARCHAR(MAX), [Type 1] NVARCHAR(MAX), [Type 2] NVARCHAR(MAX), [Total] BIGINT, [Attack] BIGINT, [Defense] BIGINT, [Speed] BIGINT, [Atk Rank] BIGINT, [Def Rank] BIGINT, [Spe Rank] BIGINT
    )
    
16

    CREATE TABLE status (
        id INT IDENTITY(1,1) PRIMARY KEY, [year] NVARCHAR(MAX), [female] FLOAT, [male] FLOAT, [single] FLOAT, [married] FLOAT, 

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 2 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLParamData)')

In [10]:
academic_df.dtypes

NameError: name 'academic_df' is not defined

In [ ]:
import pyodbc
import pandas as pd

# Connection string
server = 'azuresqledge'
database = 'datasets'
username = 'sa'
password = 'YourStrongPassword'

conn_str = (
    f'DRIVER={{ODBC Driver 18 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password};'
    'TrustServerCertificate=yes;'
    'Encrypt=yes;'
)

# Create DataFrame (assuming you have this data in a CSV file)
df = pd.read_csv("path/to/your/csv_file.csv")

try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Create table
    create_table_query = """
    CREATE TABLE international_students (
        id INT IDENTITY(1,1) PRIMARY KEY,
        year VARCHAR(7),
        students INT,
        us_students FLOAT,
        undergraduate FLOAT,
        graduate FLOAT,
        non_degree FLOAT,
        opt FLOAT
    )
    """
    cursor.execute(create_table_query)
    conn.commit()
    print("Table created successfully.")

    # Insert data
    for index, row in df.iterrows():
        insert_query = """
        INSERT INTO international_students (year, students, us_students, undergraduate, graduate, non_degree, opt)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """
        cursor.execute(insert_query, 
                       row['year'], 
                       row['students'], 
                       row['us_students'], 
                       row['undergraduate'], 
                       row['graduate'], 
                       row['non_degree'], 
                       row['opt'])
    
    conn.commit()
    print("Data inserted successfully.")

except pyodbc.Error as e:
    print(f"An error occurred: {e}")

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()